# Dimensionsreduktion

**Hinweis 1:** In der Vorlesung wird die Datenmatrix $\boldsymbol X$ so eingeführt, dass jede Zeile ein Feature repräsentiert und jede Spalte einen Datenpunkt. Mit der Singulärwertzerlegung $\boldsymbol X = \boldsymbol U \boldsymbol S \boldsymbol V^H$ führt dies zur Formel $\boldsymbol Y = \boldsymbol\Gamma^T\boldsymbol X$. In Aufgabe 1 wird dieselbe Repräsentation der Daten gewählt. 

Die Repräsentation von digitalen Daten ist allerdings häufig anders herum, jede Spalte steht dann für ein Feature und jede Zeile für einen Datenpunkt. Dies ist auch in Aufgabe 2 und 3 bei der Verwendung echter Daten der Fall. Daher ist es für diese Aufgaben notwendig, die Datenmatrix zu transponieren.

**Hinweis 2:** In der Vorlesung wird die PCA am Beispiel eines EKG-Signals eingeführt. Im Gegensatz dazu verwenden wir in Aufgabe 2 einen Datensatz, der aus verschiedensten Features besteht. Diese Features unterscheiden sich alle in ihrer Skalierung. Man muss daher die Features nicht nur mittelwertfrei machen, sondern auch noch auf eine Standardabweichung von 1 bringen. Ansonsten würde die Einheit der Features eine Rolle spielen; zum Beispiel hätte eine Strecke, die in mm angegeben wird dann einen sehr viel Größeren Einfluss als eine gleichlange Strecke, die in km angegeben wird.

## 1. Matrizenrechnung


Gegeben sei die Matrix 
$\boldsymbol X=\begin{bmatrix} 1 & -1 & 0\\ 2 & 1 & -3\end{bmatrix} \in\mathbb{R}^{2\times 3}$. 

**a)** Berechne per Hand die Eigenwerte und Eigenvektoren der Matrizen $\boldsymbol{X}\boldsymbol X^H$ und $\boldsymbol{X}^H\boldsymbol X$ und gib die Eigenwertzerlegungen an.

**b)** Berechne jetzt die Kovarianz von $\boldsymbol X$: $Cov[\boldsymbol X] = \boldsymbol\Sigma = E\Big[(\boldsymbol X-E[\boldsymbol X]) (\boldsymbol X-E[\boldsymbol X])^H\Big]$ 

**c)** Bestimme die Eigenwertzerlegung der in b) berechneten Kovarianz und gib die Ladungsmatrix $\boldsymbol\Gamma$ der Matrix $\boldsymbol X$ an. Überprüfe Dein Ergebnis, indem Du mit Python die Singulärwertzerlegung von $\boldsymbol X$ berechnen. Was stellst du fest?

##2. Principal Component Analysis (PCA)

Die Hauptkomponentenanalyse oder Principal Component Analysis (PCA) ist ein Verfahren zur Strukturierung von großen Datensätzen mit vielen Features. Diese Strukturierung kann in erster Linie für zwei Anwendungen genutzt werden:

*   Vereinfachung der Klassifikation von Daten 
*   Veranschaulichung höherdimensionaler Matrizen

Im Folgenden werden wir eine PCA auf einem Datensatz zu chronischen Nierenerkrankungen durchführen (chronic kidney diseases, *ckd*). Die 26 Spalten enthalten 24 Features, eine ID und eine Klassifikation. Jede Zeile steht für eine untersuchte Person. Die Quelle des Datensatzes und weitere Informationen zu den Features findest Du [hier](https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease).

Führe die folgenden zwei Zellen aus, um den Datensatz `kidney_disease.csv` zu importieren und zu bereinigen. Die Bereinigung beinhaltet die Löschung aller Zeilen mit mindestens einem `NaN`-Wert, sowie der Spalte `id`, die keinen Informationsgehalt hat. Außerdem werden die kategorischen Features (z.B. das Feature `appet`, das den Appetit der Person als *good* oder *poor* beschreibt), in numerische Werte transformiert. Zuletzt wird das Dataframe `kidney_dataset` noch in die Featurematrix `X` und den Zielvektor `true_label` aufgeteilt.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
kidney_dataset = pd.read_csv('/content/gdrive/MyDrive/kidney_disease.csv')
kidney_dataset

In [ ]:
# Bereinigung des Datensatzes von NaN-Werten und der Spalte id
kidney_dataset = kidney_dataset.dropna()
kidney_dataset = kidney_dataset.drop(columns='id')

# Umwandlung der kategorischen Features in Boolean mithilfe von Dictionaries und der replace-Funktion von Pandas
# rbc, pc: abnormal = 1, normal = 0
trans1={'abnormal': True, 'normal': False}
# pcc, ba: present = 1, notpresent = 0
trans2={'present': True, 'notpresent': False}
# htn, dm, cad, pe, ane: yes =1, no = 0
trans3={'yes': True, 'no': False}
# appet: good=1, poor = 0
trans4={'good': True, 'poor': False}
kidney_dataset[['rbc','pc']]=kidney_dataset[['rbc','pc']].replace(trans1)
kidney_dataset[['pcc','ba']]=kidney_dataset[['pcc','ba']].replace(trans2)
kidney_dataset[['htn','dm','cad','pe','ane']]=kidney_dataset[['htn','dm','cad','pe','ane']].replace(trans3)
kidney_dataset['appet']=kidney_dataset['appet'].replace(trans4)

# Erstellen eines eigenen Dataframes für die wahren labels
true_label = kidney_dataset['classification']
# Löschen der wahren labels aus dem Featureframe -> die wahren labels sollen nicht weiter vorverarbeitet werden!
X=kidney_dataset.drop(columns='classification')

kidney_dataset

Zur Berechnung der PCA muss jedes Feature standardisiert werden. Das heißt, dass die Features mittelwertfrei sein und eine Standardabweichung von 1 haben müssen - man nimmt also an, dass sie standardnormalverteilt sind. In unserem Fall heißt das, dass Mittelwert und Standardabweichung für jede **Spalte** der Matrix $\boldsymbol X$ berechnet und normiert werden müssen, denn jede Spalte steht für ein Feature - anders als in der VL oder Aufgabe 1! Dies Skalierung könnt ihr beispielsweise mit der Bibliothek `sklearn` machen, die dafür über den `StandardScaler` verfügt. Genausogut könnt ihr aber auch einfach Mittelwerte berechnen und subtrahieren, sowie die Standardabweichung berechnen und durch sie teilen. Es ergibt sich die Matrix $\tilde{\boldsymbol X}$. 

**Achtung:** Mehrere der Spalten sind vom Typ `Object`, was keine manuelle Rechnung zulässt (der `StandardScaler` hingegen hat damit keine Probleme). Mit dem Befehl `pd.to_numeric` kann man sie in einen numerischen Datentyp ändern. 

**a)** Erstelle das DataFrame `X_tilde`, das die standardisierten Features enthält!

**b)** Führe dann die PCA mit zwei Komponenten aus. Füge das erhaltene DataFrame mit den *true labels* zusammen und erstelle einen zweidimensionalen Scatterplot der Klassifikation über den Komponenten. Zum Plotten stellen wir euch in der folgenden Zelle die Funktion `draw_pca_scatterplot(pca_dataframe, figax, pal=palette)` zur Verfügung, die ihr verwenden könnt.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
palette = sns.color_palette("colorblind")

def draw_pca_scatterplot(pca_dataframe, figax, pal=palette):
  """
  Zeichne einen Scatterplot des Dataframes pca_dataframe

  :param pca_dataframe: Zu plottendes Dataframe, muss 2 Features mit Namen 'principal component 1' und 'principal component 2' und das Label mit Namen 'classification' enthalten;
                        falls classifier=True, muss außerdem eine Spalte 'prediciton' vorhanden sein
  :param figax: Die Figure oder Achse, auf der zu plotten ist
  :param pal: Die Farbpalette, mit der geplottet werden soll
  """
  targets = ['ckd','notckd']
  colors = [pal[0],pal[1]]

  figax.set_xlabel('Principal Component 1', fontsize = 15)
  figax.set_ylabel('Principal Component 2', fontsize = 15)
  for target, color in zip(targets,colors):
      indicesToKeep = pca_dataframe['classification'] == target
      figax.scatter(pca_dataframe.loc[indicesToKeep, 'principal component 1'], pca_dataframe.loc[indicesToKeep, 'principal component 2'], color = color, s = 50)
  figax.legend(targets)
  figax.grid()


## 3. k-Nearest-Neigbors

Zum Abschluss wollen wir die PCA nutzen, um eine einfache kNN-Klassifikation durchzuführen. 

**a)** Führe die nächsten beiden Zellen aus, um den `KNeighborsClassifier` der Bibliothek `sklearn` zu importieren und das per `sklearn` erstellte Dataframe aufzuteilen in einen Teil der fürs Training verwendet wird und einen fürs Testen.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
import random as rd
rd.seed(0)

In [ ]:
train_fraction = 0.8

train_index = np.sort(rd.sample(range(0,158),int(train_fraction*len(principal_dataframe))))
train_dataframe = principal_dataframe.iloc[train_index]
train_truth = true_label.iloc[train_index]
combined_trainframe = pd.concat([train_dataframe,train_truth],axis=1)

test_mask = np.ones(principal_dataframe.shape[0],dtype=bool)
test_mask[train_index] = False
test_dataframe = principal_dataframe.iloc[test_mask]
test_truth = true_label.iloc[test_mask]

**b)** Trainiere jetzt einen kNN-Klassifizierer mit den Trainingsdaten und teste ihn an den Testdaten. Probiere verschiedene Werte für *k* aus und schaue dir die Ergebnisse an. Verwende den eben importierten Klassifizierer `KNeighborsClassifier`.